In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector

In [2]:
cnx = mysql.connector.connect(user='root', password='Kh18riku!',
                                host='127.0.0.1',
                                database='global_n2o')
cursor = cnx.cursor()

query = ("SELECT * FROM LinearMeasurement")
# Execute the query and fetch all results
rowdata = pd.read_sql(query, cnx)
# Close the connection
cnx.close()

# Drop rows where dbdata['HMTruth'] == None or -1
rowdata = rowdata.dropna(subset=['HMTruth'])
rowdata = rowdata[rowdata['HMTruth'] != -1]
rep_ids = rowdata['RepID'].unique()

/home/ryan/.virtualenv/time/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
# Create a dataframe with RepIDs and Verification columns
verification_df = pd.DataFrame(columns=['RepID', 'Verification'])
verification_df['RepID'] = rep_ids
verification_df['Verification'] = None
%matplotlib tk
# Get completed Rep IDs from verification.csv
verification_df = pd.read_csv('verification.csv')
completed_ids = verification_df[verification_df['Verification'].notnull()]['RepID'].unique()
# Select only the RepIDs that have not been completed
rep_ids = np.setdiff1d(rep_ids, completed_ids)
# Loop through each RepID and create a scatterplot from rowdata
for rep_id in rep_ids:
    # Clear the current figure
    plt.clf()
    # Create a dataframe for the current rep_id
    rep_df = rowdata[rowdata['RepID'] == rep_id]
    # For each row, if N2OFlux is less than 5, set HMTruth to 0
    rep_df.loc[rep_df['N2OFlux'] < 5, 'HMTruth'] = 0
    # Create a scatterplot of the current rep_id
    # For each point, if HMTruth is 1, color it red. if HMTruth is 0, color it blue
    plt.scatter(rep_df['Date'], rep_df['N2OFlux'], c=rep_df['HMTruth'].apply(lambda x: 'red' if x == 1 else 'blue'))
    plt.xlabel('Date')
    plt.ylabel('Flux')
    plt.title('RepID: ' + str(rep_id))
    # Show the plot in a new popup window
    plt.show(block=False)
    # Wait for the plot to be closed
    plt.waitforbuttonpress()

    # Ask user if the data is good. Include a Yes, No, and Quit button option. Include the repid in the question
    verification = input('Is the data good for RepID ' + str(rep_id) + '? (y/n/q): ')
    # If the data is good, set the verification column to True
    if verification == 'y':
        verification_df.loc[verification_df['RepID'] == rep_id, 'Verification'] = True
    # If the data is bad, set the verification column to False
    elif verification == 'n':
        verification_df.loc[verification_df['RepID'] == rep_id, 'Verification'] = False
    # If the user wants to quit, break out of the loop
    elif verification == 'q':
        break
    # If the user enters something other than y or n, ask again
    else:
        verification = input(str(rep_id) + ': Please enter y, n, or q: ')
        if verification == 'y':
            verification_df.loc[verification_df['RepID'] == rep_id, 'Verification'] = True
        elif verification == 'n':
            verification_df.loc[verification_df['RepID'] == rep_id, 'Verification'] = False
        elif verification == 'q':
            break
    # Close the current figure
    plt.close()
    # Save the verification_df to a csv file
    verification_df.to_csv('verification.csv', index=False)